In [25]:
from bs4 import BeautifulSoup
import requests as rq
import json
import pandas as pd
import numpy as np
import re

In [30]:
file_path = "processed_data_that_scrapped_with_scrapy_test.csv"
df = pd.read_csv(file_path)

In [31]:
new_df = df[df["review_text"].isna()]

In [32]:
nan_indices = new_df.index
len(nan_indices)

5668

In [34]:
def scrapy(df,nan_indices):
    # review_texts = []
    # review_values = []
    for index in nan_indices:
        url = df.loc[index]["Link"]
        # print(url)
        try:
            response = rq.get(url)

            if response.status_code == 200:
                soup = BeautifulSoup(response.text, "lxml")

                name = soup.find("div", id='appHubAppName', class_ = 'apphub_AppName').text
                parent_div = soup.find('div', class_ = "summary_section")
                review_summary = parent_div.find_all('span')

                span_text = ""
                span_value = ""

                for i, span in enumerate(review_summary):
                    if i ==0:
                        span_text = span.text.strip()
                    elif i==1:
                        span_value = re.sub(r'[^\d,]', '', span.text.strip())

                # review_texts.append(span_text)
                # review_values.append(span_value)
                # print("response 200 döndü.")
                df.loc[index,"review_text"] = span_text
                df.loc[index, "review_value"] = span_value
            else:
                # Hata durumunda NaN ekleyelim
                print(url,"response 200 dönmedi.")
                df.loc[index,"review_text"] = np.nan
                df.loc[index, "review_value"] = np.nan
        except Exception as e:
            print(f"Hata oluştu: {e}")
            # # Hata durumunda boş değerler ekle
            # review_texts.append(None)
            # review_values.append(None)

    # data["Review Text"] = review_texts
    # data["Review Value"] = review_values

    return df

In [ ]:
df_new = scrapy(df, nan_indices)

In [37]:
df_new.isna().sum()

Title                         0
Original Price                0
Discounted Price              0
Release Date                  0
Link                          0
Game Description            148
Recent Reviews Summary     5419
All Reviews Summary       56268
Recent Reviews Number      5419
All Reviews Number        56268
Developer                   171
Publisher                   399
Supported Languages           0
Popular Tags                  0
Game Features                 0
Minimum Requirements       1047
Memory_mb                  4593
Storage_mb                 5563
review_text                3149
review_value               3149
dtype: int64

In [39]:
df_new.to_csv("processed_data_scrapped_with_scrapy_and_beautifulsoup.csv", index = False)